# Map Page

> The page of the app that generates the map view of the data.  

In [ ]:
#| default_exp pages/map_page

In [ ]:
#| export
# TODO minimise these imports a bit where possible
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.dash_components import gen_bar_plot, gen_dd
    from SolomonIslandsDataMap.app_data import mytitle, map_graph, selectedBarGraph, fake_slider, stored_data
    from SolomonIslandsDataMap.load_data import SolomonGeo
except: 
    from dash_components import gen_bar_plot, gen_dd
    from app_data import mytitle, map_graph, selectedBarGraph, fake_slider, stored_data
    from load_data import SolomonGeo
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import numpy as np
from fastcore.test import *
from dash import Dash, callback, clientside_callback, dcc, Output, Input, State, html, Patch, ctx, register_page  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
from dash.exceptions import PreventUpdate
import dash_mantine_components as dmc
import os
import time
import json

ModuleNotFoundError: No module named 'dash_components'

In [ ]:
#| export
# Try catch is added here so that the notebook can be tested without the app being run
try:
    register_page(__name__, 
                        path='/',
                        title='Census Map',
                        name='Census Map')
except:
    pass

In [ ]:
#| export
# Creat some hacked init things
# TODO again as below, must be a better way
init_init = dcc.Store(id="initial-initial", data='census')

Setup some of  the key data contiaining objects

## Customize the Layout

In [ ]:
#| export
def layout():
    return  mytitle, \
                dcc.Loading(
                    id="loading-map",
                    type="default",
                    children = map_graph,
                ),\
                dcc.Loading(
                    id="loading-bar-graph",
                    type="default",
                    children = selectedBarGraph,), \
                init_init, fake_slider

## Global Variables

In [ ]:
#| export
dict_sol = stored_data.data

# Build callbacks to create interaction

## Manage Sidebar
On page load, manage the dropdowns displayed in the sidebar for all pages

#| export
@callback(
    Output("segmented_geo", "value"),
    Output('segmented_geo', 'disabled'), # On page load, allow for changing geography
    Output("dataset-html", "style"),
    Output("age-html", "style"),
    Output("rows-html", "style"),
    Output("census-vars-html", "style"),
    Output("pop-vars-html", "style"),
    Input("dataset_type", "value"),
    Input("initial-initial", 'data'),
    State("segmented_geo", "value"),
)
def maintain_sidebar(dataset:str, # Currently selected dataset
                        page_trigger:str, # Page that triggered initial load
                        geo:str, # the current geo level selection
                         ) -> dict:
    """Manages the dropdowns actively visable in sidebar based on page loaded"""
    print("Func: maintain_sidebar")
    # Depending on the page loaded, geogrpahy will or will not be disabled
    geo_disable = False

    # Based on page, update hidden style
    hide = {'display': 'none'}
    show = {'display': 'block'} 
    displayDataset = ''
    if page_trigger == 'census':
        displayDataset = hide
        displayAges = hide
        displayRows = hide
        censusVars = show
        popVars = hide
    elif page_trigger == 'pop':
        displayDataset = hide
        displayAges = show
        displayRows = hide
        # Disable geo selection on population page and set geo to province
        geo_disable = True 
        censusVars = hide
        popVars = show
        if geo != 'Province':
            # When the initial load id triggered by navigation to population page, 
            # if the geo isn't province we reset to this
            geo = 'Province'

    elif page_trigger == 'table':
        displayDataset = show
        displayAges = hide
        displayRows = show
        if dataset == 'Census':
            censusVars = show
            popVars = hide
        elif dataset == 'Population Projections':
            censusVars = hide
            popVars = show
        else:
            ValueError("Dataset must be Census or Population")


    if dataset == 'Population Projections':
        geo_disable = True

    return geo, geo_disable, displayDataset, displayAges, displayRows, censusVars, popVars

In [ ]:
#| export
clientside_callback(
    """
    function maintain_sidebar(dataset, page_trigger, geo)
    {
        //Manages the dropdowns actively visable in sidebar based on page loaded
        console.log("Func: maintain_sidebar");
        // Depending on the page loaded, geogrpahy will or will not be disabled
        var geo_disable = false;

        // Based on page, update hidden style
        var hide = {'display': 'none'};
        var show = {'display': 'block'};

        var defualtLocation = show;
        var electionLocation = hide;
        var displayDataset = '';
        if (page_trigger === 'census')
        {
            displayDataset = hide;
            displayAges = hide;
            displayRows = hide;
            displayElections = hide;
            censusVars = show;
            popVars = hide;
        } else if (page_trigger === 'pop')
        {
            displayDataset = hide;
            displayAges = show;
            displayRows = hide;
            displayElections = hide;
        // Disable geo selection on population page and set geo to province
            geo_disable = true; 
            censusVars = hide;
            popVars = show;
            if (geo !== 'Province')
            {
                // When the initial load id triggered by navigation to population page, 
                // if the geo isn't province we reset to this
                geo = 'Province';
            };
        } else if (page_trigger === 'table')
        {
            displayDataset = show;
            displayAges = hide;
            displayRows = show;
            displayElections = hide; //TODO should add this into the dataset eventually
            if (dataset === 'Census')
            {
                censusVars = show;
                popVars = hide;
            } else if (dataset === 'Population Projections')
            {
                censusVars = hide;
                popVars = show;
            } else
            {
                console.log("Dataset must be Census or Population")
            };
                
        } else if (page_trigger === 'election')
        {
            displayDataset = hide;
            displayAges = hide;
            displayRows = hide;
            displayElections = show;
            geo_disable = true; 
            censusVars = hide;
            popVars = hide;
            defualtLocation = hide;
            electionLocation = show;
            if (geo !== 'Constituency')
            {
                // When the initial load id triggered by navigation to election page, 
                // if the geo isn't constituency we reset to this
                geo = 'Constituency';
            };
                
        };


        if (dataset == 'Population Projections')
        {
            geo_disable = true;
        };

        return [geo, geo_disable, displayDataset, displayAges, displayRows, censusVars, popVars, displayElections, defualtLocation, electionLocation];
    };
    """,
    Output("segmented_geo", "value"),
    Output('segmented_geo', 'disabled'), # On page load, allow for changing geography
    Output("dataset-html", "style"),
    Output("age-html", "style"),
    Output("rows-html", "style"),
    Output("census-vars-html", "style"),
    Output("pop-vars-html", "style"),
    Output("elections-dd-html", "style"),
    Output('location-html', "style"),
    Output('location-election-html', "style"),
    Input("dataset_type", "value"), # Currently selected dataset
    Input("initial-initial", 'data'), # Page that triggered initial load
    State("segmented_geo", "value"), # the current geo level selection
)
# TODO - clear should be false and set value as central honiara say
# TODO - or do I make a callback that just inputs a new dropdown in the dropdown with better settings?

Update active page on nav based on current page

In [ ]:
#| export
@callback(
    Output('censusNav', 'active'),
    Output('electionNav', 'active'),
    Output('popNav', 'active'),
    Output('tableNav', 'active'),
    Input("initial-initial", 'data'), # Page that triggered initial load
    prevent_initial_call=True,
    allow_duplicate=True,
) # TODO - eventually can I turn this into a clientside callbback?
def navClick(clickData:str, # CUrrently selected map page 
                )->[str]: # Returns the active navlink
    """This function updates the actively selected page on the navbar"""

    if clickData == 'census':
        censusActive = True
        electionActive = False
        popActive = False
        tableActive = False
    if clickData == 'election':
        censusActive = False
        electionActive = True
        popActive = False
        tableActive = False
    if clickData == 'pop':
        censusActive = False
        electionActive = False
        popActive = True
        tableActive = False
    if clickData == 'table':
        censusActive = False
        electionActive = False
        popActive = False
        tableActive = True

    return censusActive, electionActive, popActive, tableActive

### Update Current Dataset
When the census or population page are loaded, then set the currently selected value
of the dataset dropdown to the respective ones

#| export
@callback(
    Output("dataset_type", "value"), # Based on page loaded, change value of current dataset
    Input("initial-initial", 'data'),
    State("dataset_type", "value"),
    allow_duplicate=True,
    prevent_initial_call=True,
)
def dataset_selection(page_trigger:str, # Newly loaded page
                      currDataset:str, # Currenly selected data
                         ) -> str: # New dataset
    """Based on the page selected, update the dataset selected"""
    print("Func: dataset_selection")
    if page_trigger == 'census':
        dataset = 'Census'
    elif page_trigger == 'pop':
        dataset = 'Population Projections'
    else:
        dataset = currDataset
    
    return dataset

show_doc(dataset_selection)

In [ ]:
#| export
clientside_callback(
    """
    function dataset_selection(page_trigger, currDataset)
    {
        //Based on the page selected, update the dataset selected
        console.log("Func: dataset_selection");
        var dataset = '';
        if (page_trigger === 'census')
        {
            dataset = 'Census';
        }
        else if (page_trigger === 'pop')
        {
            dataset = 'Population Projections';
        } else
        {
            dataset = currDataset;
        };
        
        return dataset;
    };
    """,
    Output("dataset_type", "value"), # Based on page loaded, change value of current dataset
    Input("initial-initial", 'data'), # Newly loaded page
    State("dataset_type", "value"), # Newly loaded page
    allow_duplicate=True,
    prevent_initial_call=True,
)

## Update Title 
For all pages of the dash app

#| export
@callback(
    Output('title', 'children'),
    Input("segmented_geo", 'value'),
    Input("segmented_type", 'value'),
    Input('measureDropdown', 'value'),
    Input('varDropdown', 'value'),
    Input('measureDropdownPop', 'value'),
    Input("year_slider", "value"),
    Input('locDropdown', 'value'),
    Input("dataset_type", 'value'),
    Input('initial-initial', 'data'),
    allow_duplicate=True,
    prevent_initial_call=True)
def update_title(geo_input:str, # User input from the geography dropdown
                data_type:str, # User input of type of data
                measure:str, # A string contiaining the census variable and measure split by ':'
                variable:str, # The state of the variable dropdown
                measurePop:str,
                year:str, # The currently selected year
                loc_selection:[str], # The selected locations, may be none
                dataset:str, # The currently selected dataset
                load_trigger:str, # The currently loaded page
              )->(str): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the title for each of the pages
    '''
    
    print("Func: update_title")
    
    title = ''
    if load_trigger == 'census':
      
      title = '## ' + data_type + " " + measure + ' by ' + geo_input
      if data_type == 'Proportion': title = '## ' + data_type + " of " + measure + ' by ' + geo_input
      
    elif load_trigger == 'pop':
      
      # Create a title for the pyramid
      if loc_selection == []:
          title = '## Projected Population Pyramid for Solomon Islands'
      else:
          title = '## Aggregated Projected Population Pyramid for ' + ', '.join(loc_selection)
      title += ' in ' + str(year)
      
    elif load_trigger == 'table':
      
      title = '## Solomon Islands Data map - ' + geo_input
      if dataset == 'Census':
        title = '## Census ' + variable + " by " + geo_input
      elif dataset == 'Population Projections':
        title = '## Projections of ' + measurePop + " population by " + geo_input
      
  
    
    return title

In [ ]:
#| export
clientside_callback(
    """
    function update_title(geo_input,
                data_type, 
                measure, 
                variable, 
                measurePop,
                year, 
                loc_selection, 
                dataset, 
                load_trigger)
    {
      //Updates the title for each of the pages
      console.log("Func: update_title");
      
      var title = '';
      if (load_trigger === 'census')
      {
        
        title = '## ' + data_type + " " + measure + ' by ' + geo_input;
        if (data_type === 'Proportion') 
        {
          title = '## ' + data_type + " of " + measure + ' by ' + geo_input;
        };
      } else if (load_trigger == 'pop')
      {
        // Create a title for the pyramid
        if (loc_selection.length === 0)
        {
            title = '## Projected Population Pyramid for Solomon Islands';
        } else
        {
            title = '## Aggregated Projected Population Pyramid for ' + loc_selection.join(', ');
            title += ' in ' + year;
        };
      } else if (load_trigger === 'table')
      { 
        title = '## Solomon Islands Data map - ' + geo_input;
        if (dataset === 'Census')
        {
          title = '## Census ' + variable + " by " + geo_input;
        } else if (dataset === 'Population Projections')
        {
          title = '## Projections of ' + measurePop + " population by " + geo_input;
        };
      };
      return title;
    };
    """,
    Output('title', 'children'),
    Input("segmented_geo", 'value'),  # User input from the geography dropdown
    Input("segmented_type", 'value'), # User input of type of data
    Input('measureDropdown', 'value'), # A string contiaining the census variable and measure split by ':'
    Input('varDropdown', 'value'), # The state of the variable dropdown
    Input('measureDropdownPop', 'value'), 
    Input("year_slider", "value"),# The currently selected year
    Input('locDropdown', 'value'), # The selected locations, may be none
    Input("dataset_type", 'value'), # The currently selected dataset
    Input('initial-initial', 'data'), # The currently loaded page
    allow_duplicate=True,
    prevent_initial_call=True)


## Update location dropdown value based on map click
This callback also will trigger an update to the selected card data

In [ ]:
#| export
@callback(
    Output('locDropdown', 'value', allow_duplicate=True),
    Output('map', "clickData"),
    Output('map', "selectedData"),
    Input('map', 'clickData'),
    Input('map', 'selectedData'),
    State('locDropdown', 'value'),
    prevent_initial_call=True,
    allow_duplicate=True,
) # TODO - eventually can I turn this into a clientside callbback?
def map_click(clickData:dict, # The currently clicked location on the map
              selectedData:dict, # The currently selected locations on the map
                prev_locs:[str], # The previously selected locations
                )->[str]: # Returns the new value for the dropdown
    """This function updates the dropdown menu based on the map click data"""

    print("Func: map_click")
    if clickData is None and selectedData is None:
        # When nothing is selected or clicked, don't update anything
        raise PreventUpdate # if changing to clientside, use: window.dash_clientside.no_update
        #return prev_locs, None, None
    else:
        # The locations are list of dictionaries
        if selectedData is not None:
            selections = list(map(lambda x: x['location'], selectedData['points']))

        elif clickData is not None:
            selections = list(map(lambda x: x['location'], clickData['points']))
        locations = []
        if prev_locs: locations = prev_locs
        # Check whether the new location is already in the prev locations
        for selection in selections:
            if selection in locations: locations.remove(selection)
            else: locations.append(selection)
    
        # returned objects are assigned to the component property of the Output
        # After updating fileter, we always reset map selection 
        return locations, None, None
        


In [ ]:
#| hide
show_doc(map_click)

---

### map_click

>      map_click (clickData:dict, selectedData:dict, prev_locs:[<class'str'>])

This function updates the dropdown menu based on the map click data

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| clickData | dict | The currently clicked location on the map |
| selectedData | dict | The currently selected locations on the map |
| prev_locs | [<class 'str'>] | The previously selected locations |
| **Returns** | **[<class 'str'>]** | **Returns the new value for the dropdown** |

#| hide
show_doc(update_measure)

## Update map selections

In [ ]:
#| export
# TODO merge back into 
@callback(
    Output('map', "figure", allow_duplicate=True),
    # TODO - make this a Row object with children, then use function to recontruct
    # a group of them
    Input('locDropdown', 'value'),
    State("segmented_geo", "value"),
    #State('geo_df', 'data'),
    prevent_initial_call=True,
    allow_duplicate=True,
)
def map_selections(locations:[str], # The previously selected locations
                geo_input, # Currently selected geography
                #dict_sol:dict, # The dataset in dictionary form
                )->[str]: # Returns the new value for the dropdown
    '''
    Update the selected data on the map for the selected locations
    Selections is an array of integers indicating the index of the selected points
    '''
    print("Func: map_selections")
    
    # Using geo from stored values
    sol_geo = SolomonGeo.gen_stored(dict_sol)
    
    patched_figure = Patch()
    
    ct = np.where(sol_geo.geo_levels == geo_input)[0][0] # Tracks the trace number
    pot_locs = map_graph.figure['data'][ct]['locations']
    
    if locations: 
        
        selections = np.nonzero(np.in1d(pot_locs, locations))[0]
        
    else: 
        
        selections = None 
        

    
    patched_figure['data'][ct]['selectedpoints'] = selections
    
    # returned objects are assigned to the component property of the Output
    # After updating fileter, we always reset map selection 
    return patched_figure
        


In [ ]:
#| hide
show_doc(map_selections)

---

### map_selections

>      map_selections (locations:[<class'str'>], geo_input)

Update the selected data on the map for the selected locations
Selections is an array of integers indicating the index of the selected points

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| locations | [<class 'str'>] | The previously selected locations |
| geo_input |  | Currently selected geography |
| **Returns** | **[<class 'str'>]** | **Returns the new value for the dropdown** |

## Update the location options based on geography selected

In [ ]:
#| export
@callback(
    Output('locationDiv', 'children'),
    Input("segmented_geo", 'value'),
    #State('geo_df', 'data'),
    State('locDropdown', 'value'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_geography(geo_input:str, # User input from the geography dropdown
                    #dict_sol:dict, # The dataset in dictionary form
                    locations:[str], # Currently selected locations
              )->[str]: # Returns a new list of locations to display
    '''
    Updates the dropdown_location dropdown based on the currently selected data aggregation.
    Check to see if current locations are in geography, if they are not then reset them.
    '''
    print("Func: update_geography")
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data
    
    # If all selected locations are in new geo, then keep old locations
    new_locations = []
    if locations != [] and set(locations) <= set(sol_geo.locations[geo_input]):
        new_locations = locations


    return gen_dd(sol_geo.locations[geo_input], 'locDropdown', "Select a location", clear = True, multi = True, 
                  val = new_locations)

In [ ]:
show_doc(update_geography)

---

### update_geography

>      update_geography (geo_input:str, locations:[<class'str'>])

Updates the dropdown_location dropdown based on the currently selected data aggregation.
Check to see if current locations are in geography, if they are not then reset them.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_input | str | User input from the geography dropdown |
| locations | [<class 'str'>] | Currently selected locations |
| **Returns** | **[<class 'str'>]** | **Returns a new list of locations to display** |

## Update the variable dropdowns based on the selection of the overall variable

In [ ]:
#| export
@callback(
    Output('measureDiv', 'children', allow_duplicate=True),
    Input('varDropdown', 'value'),
    #State('geo_df', 'data'),
    State('measureDropdown', 'value'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_measure(new_var:str, # Selected variable
                   #dict_sol:dict, # The dataset in dictionary form
                   measure:str, # Currently selected measure
              )->dcc.Dropdown: # Returns a dropdown of measures for selected variable
    '''
    Updates the dropdown_location dropdown based on the currently selected data aggregation.
    '''
    print("Func: update_measure")
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data

    # Sometimes this callback is triggered when the measure doesn't need to be reset.
    # Check whether measure is in variable, if not reset to 0
    if measure not in sol_geo.census_vars[new_var]:
        measure = sol_geo.census_vars[new_var][0]

    # When a variable is selected, the measure will be set as the first one
    return gen_dd(sol_geo.census_vars[new_var], 'measureDropdown', 
                  val = measure)

In [ ]:
#| hide
show_doc(update_measure)

---

### update_measure

>      update_measure (new_var:str, measure:str)

Updates the dropdown_location dropdown based on the currently selected data aggregation.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| new_var | str | Selected variable |
| measure | str | Currently selected measure |
| **Returns** | **Dropdown** | **Returns a dropdown of measures for selected variable** |

## Update the measure selection from clicks on the bar graph

In [ ]:
#| export
@callback(
    Output('measureDropdown', 'value', allow_duplicate=True),
    Output('bar_graph', "clickData"),
    Input('bar_graph', 'clickData'),
    #State('varDropdown', 'value'),
    #State('geo_df', 'data'),
    prevent_initial_call=True,
    allow_duplicate=True,
)
def bar_click(clickData:dict, # The currently clicked location on bar graph
                )->[str]: # Returns the new value for the dropdown
    """This function updates the dropdown menu based on the bar graph click data"""
    print("Func: bar_click")
    if clickData is None:
        # Trying to prvent update when nothing is clicked
        raise PreventUpdate
        #return sol_geo.census_vars[variable][0], None
    else:
        # The measure are list of dictionaries
        selection = list(map(lambda x: x['x'], clickData['points']))[0]
        print(selection)
    
        # returned objects are assigned to the component property of the Output
        # After updating fileter, we always reset map selection 
        return selection, None
        


In [ ]:
#| hide
show_doc(bar_click)

---

### bar_click

>      bar_click (clickData:dict)

This function updates the dropdown menu based on the bar graph click data

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| clickData | dict | The currently clicked location on bar graph |
| **Returns** | **[<class 'str'>]** | **Returns the new value for the dropdown** |

#| export
clientside_callback(
    """function bar_click(clickData,
                variable,
                dict_sol)
    {
        //This function updates the dropdown menu based on the bar graph click data
        console.log("Func: bar_click");
        if (typeof clickData === 'undefined')
        {
            // Trying to prvent update when nothing is clicked
            return window.dash_clientside.no_update;
        } else 
        {
            // The measure are list of dictionaries
            var t2 = clickData.points;
            var selection = [];
            for(var i = 0; i < t2.length; i++)
            {
                selection.push(t2[i]['location']);
            };
        
            # returned objects are assigned to the component property of the Output
            # After updating fileter, we always reset map selection 
            return [selection[0], undefined];
        };
    };
    """,
    Output('measureDropdown', 'value', allow_duplicate=True),
    Output('bar_graph', "clickData"),
    Input('bar_graph', 'clickData'),# The currently clicked location on bar graph
    State('varDropdown', 'value'), # The currently selected variable
    State('geo_df', 'data'), # The dataset in dictionary form
    prevent_initial_call=True,
    allow_duplicate=True,
)

        

# Update map based on inputs
This updates the map for both the census page and the population page

In [ ]:
#| export
@callback(
    Output('map', 'figure', allow_duplicate=True),
    Input("segmented_geo", 'value'),
    Input("segmented_type", 'value'),
    Input('measureDropdown', 'value'),
    Input('varDropdown', 'value'),
    Input('measureDropdownPop', 'value'),
    Input('varDropdownPop', 'value'),
    Input('age_dropdown', 'value'),
    Input("year_slider", "value"),
    Input('segmented_geo', 'disabled'),
    Input('initial-initial', 'data'),
    #State('geo_df', 'data'),
    allow_duplicate=True,
    prevent_initial_call=True)
def update_map(geo_input:str, # User input from the geography dropdown
                data_type:str, # User input of type of data
                measure:str, # A string contiaining the census variable and measure split by ':'
                variable:str, # The state of the variable dropdown
                measurePop:str, # A string contiaining the census variable and measure split by ':'
                variablePop:str, # The state of the variable dropdown
                age:[str], # Age Brackets to display
                year:int, # The selected projection year
                geo_trigger:int, # Listening for whether segmented geo is locked, signaling that map
                                        # page was loaded after geo data was updated.
                page:str, # The current page
                #dict_sol:dict, # The dataset in dictionary form
              )->(type(go.Figure())): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    
    # TODO the None workaround might be taxing on the load times, is there a better way
    # or maybe I can check it it needs updating?
    patched_figure = Patch()
    button_clicked = ctx.triggered_id
    print("Func: update_map")
    print("Updating map for " + page)
    
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data
    
    if page == 'census':
        '''Process of updating map when the selected page is census'''

        # A None value is passed when the page is first loaded, hence
        # the the values are reset.
        if button_clicked in ['segmented_geo', 'initial-initial']:
            # Update disaplayed geography 
            
            for geo in sol_geo.geo_levels:
                tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
                patched_figure['data'][tn]['visible'] = geo_input == geo
                
            
        if button_clicked in ["segmented_type", 'initial-initial']:
            # Update the type of data displayed on map and the hover template
            
            for geo in sol_geo.geo_levels:
                tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
                ar = sol_geo.get_census(geo_filter = geo, type_filter=data_type, var = variable, measure = measure).values
                ar = ar.reshape((ar.shape[0],))
                if data_type == 'Total':
                    ht = '%{customdata} <extra>%{z}</extra>'
                elif data_type == 'Proportion':
                    ht = '%{customdata} <extra>%{z:.1%}</extra>'
                else:
                    ValueError("Data type of map not recognised and note accounted for")
                patched_figure['data'][tn]['z'] = ar
                patched_figure['data'][tn]['zmin'] = np.min(ar)
                patched_figure['data'][tn]['zmax'] = np.max(ar)
                patched_figure['data'][tn]['hovertemplate'] = ht
                


        if button_clicked in ['measureDropdown', 'initial-initial']:
            
            # Update the z values in map to the data for the requested census variable
            for geo in sol_geo.geo_levels:
            # Ar updates the z value ie. data disaplyed each time
            # TODO this is fairly inefficient, as we are processing each time
            # Maybe faster framework like polars could help? or caching but would require a lot of caching
                tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
                ar = sol_geo.get_census(geo_filter = geo, type_filter=data_type, var = variable, measure=measure).values
                ar = ar.reshape((ar.shape[0],))
                patched_figure['data'][tn]['z'] = ar
                patched_figure['data'][tn]['zmin'] = np.min(ar)
                patched_figure['data'][tn]['zmax'] = np.max(ar)
            

    elif page == 'pop':
        '''Process of updating map when the selected page is population projections'''
        init_load = False
        if geo_trigger == True: init_load = True

        sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data

        # A None value is passed when the page is first loaded, hence
        # the the values are reset.
        # Hardcoded to province as we only have forcasts by province
        if button_clicked in ["segmented_geo"] or init_load == True:
            # Update disaplayed geography 
            for geo in sol_geo.geo_levels:
                tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
                patched_figure['data'][tn]['visible'] = geo_input == geo
            
        if button_clicked in ["segmented_type"] or init_load == True:
            # Update the type of data displayed on map and the hover template
            for geo in sol_geo.geo_levels:
                tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
                # All years allows us to set the min and max colour as the min and max across all years
                all_years = sol_geo.get_pop(years = sol_geo.pop_years, var = variablePop, measure = measurePop, type_filter=data_type,
                                    agg = True, agg_location = True, ages = age)
                ar = all_years.loc[year].values[:, -1]
                all_years = all_years.values[:, -1]
                ar = ar.reshape((ar.shape[0],))
                if data_type == 'Total':
                    ht = '%{customdata} <extra>%{z}</extra>'
                elif data_type == 'Proportion':
                    ht = '%{customdata} <extra>%{z:.1%}</extra>'
                else:
                    ValueError("Data type of map not recognised and note accounted for")
                patched_figure['data'][tn]['z'] = ar
                patched_figure['data'][tn]['zmin'] = np.min(all_years)
                patched_figure['data'][tn]['zmax'] = np.max(all_years)
                patched_figure['data'][tn]['hovertemplate'] = ht

                
            

        if button_clicked in ['measureDropdownPop', 'year_slider', 'age_dropdown'] or init_load == True:
            # Update the z values in map to the data for the requested census variable
            for geo in sol_geo.geo_levels:
            # Ar updates the z value ie. data disaplyed each time
            # TODO this is fairly inefficient, as we are processing each time
            # Maybe faster framework like polars could help? or caching but would require a lot of caching
                tn = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
                
                # All years allows us to set the min and max colour as the min and max across all years
                all_years = sol_geo.get_pop(years = sol_geo.pop_years, var = variablePop, measure = measurePop, type_filter=data_type,
                                    agg = True, agg_location = True, ages = age)
                ar = all_years.loc[year].values[:, -1]
                all_years = all_years.values[:, -1]
                ar = ar.reshape((ar.shape[0],))
                patched_figure['data'][tn]['z'] = ar
                patched_figure['data'][tn]['zmin'] = np.min(all_years)
                patched_figure['data'][tn]['zmax'] = np.max(all_years)

    # returned objects are assigned to the component property of the Output
    # After updating fileter, we always reset map selection 

    return patched_figure

In [ ]:
#| hide
show_doc(update_map)

---

### update_map

>      update_map (geo_input:str, data_type:str, measure:str, variable:str,
>                  measurePop:str, variablePop:str, age:[<class'str'>],
>                  year:int, geo_trigger:int, page:str)

Updates the focus census variable or geography dispalayed on the map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_input | str | User input from the geography dropdown |
| data_type | str | User input of type of data |
| measure | str | A string contiaining the census variable and measure split by ':' |
| variable | str | The state of the variable dropdown |
| measurePop | str | A string contiaining the census variable and measure split by ':' |
| variablePop | str | The state of the variable dropdown |
| age | [<class 'str'>] | Age Brackets to display |
| year | int | The selected projection year |
| geo_trigger | int | Listening for whether segmented geo is locked, signaling that map |
| page | str | The current page |
| **Returns** | **Figure** | **Returns a graph object figure after being updated and the dynamic title** |

### Update the selected part of the map based on the dropdown

### Update Selected Bar Graph based on inputs

In [ ]:
#| export
# Callback allows components to interact
@callback(
    Output('bar_graph', 'figure'),
    Input("segmented_type", 'value'),
    Input('measureDropdown', 'value'),
    Input('locDropdown', 'value'),
    Input('initial-initial', 'data'),
    State("segmented_geo", 'value'),
    State('varDropdown', 'value'),
    #State('geo_df', 'data'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_bargraph(data_type:str, # User input of type of data
                     measure:str, # A string contiaining the census variable and measure split by ':'
                     loc_selection:[str], # The selected locations, may be none
                     init_load:{}, # An empty dictionary always, triggers initial load
                     geo_input:str, # User input from the geography dropdown
                     variable:str, # The state of the variable dropdown
                     #dict_sol:dict, # The dataset in dictionary form
              )->(type(go.Figure())): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    sol_geo = SolomonGeo.gen_stored(dict_sol) # reload the data

    # Create newly selected barplot
    print("Func: update_bargraph")
    locs = []
    # Multi dropdown can return None or a list of None.
    if len(loc_selection) > 0: 
        locs = loc_selection
    bg = gen_bar_plot(sol_geo, geo_input, variable, measure, locs, data_type)

    # returned objects are assigned to the component property of the Output
    # After updating fileter, we always reset map selection 

    return  bg

On click, update the contents of the kpi's

In [ ]:
#| hide
show_doc(update_geography)

---

### update_geography

>      update_geography (geo_input:str, locations:[<class'str'>])

Updates the dropdown_location dropdown based on the currently selected data aggregation.
Check to see if current locations are in geography, if they are not then reset them.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_input | str | User input from the geography dropdown |
| locations | [<class 'str'>] | Currently selected locations |
| **Returns** | **[<class 'str'>]** | **Returns a new list of locations to display** |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()